In [ ]:
#TODO add wind bearing relative to rider
#TODO consider #pip install gpxpy or just implement the maths
#TODO create class based structure for the data and functions
#imports
import numpy as np
import tcxparser
import csv,json,requests,os
from pprint import pprint

In [135]:
class tcxWeather:
    
    
    def __init__(self, xmlfile='demoRoute.tcx'):
        self.raw = tcxparser.TCXParser(xmlfile)
        self.stravaTime = self.raw.time_values()
        self.length = len(self.stravaTime)
        self.latitude = self.raw.latitude_points()
        self.longitude = self.raw.longitude_points()
        self.distance = self.raw.distance_points()
        self.__bearing__()      
            
            
    def __bearing__(self):
        self.bearing = list()
        self.bearing.append(0) #first bearing 0
        φ = list()
        λ = list()
        for x in self.latitude:
            φ.append(np.deg2rad(x))
        for x in self.longitude:
            λ.append(np.deg2rad(x))
        
        for x in range(1,self.length):
            y = np.sin(λ[x]-λ[x-1]) * np.cos(φ[x])
            x = np.cos(φ[x-1])*np.sin(φ[x]) - np.sin(φ[x-1])*np.cos(φ[x])*np.cos(λ[x]-λ[x-1])
            self.bearing.append(np.degrees(np.arctan2(y, x))%360)
       
    
    def __bearingdec__(self): #fix to one function for both bearings
        self.bear = list()
        self.bear.append(0) #first bearing 0
        φ = list()
        λ = list()
        for x in self.lat:
            φ.append(np.deg2rad(x))
        for x in self.lon:
            λ.append(np.deg2rad(x))
        
        for x in range(1,len(self.lat)):
            y = np.sin(λ[x]-λ[x-1]) * np.cos(φ[x])
            x = np.cos(φ[x-1])*np.sin(φ[x]) - np.sin(φ[x-1])*np.cos(φ[x])*np.cos(λ[x]-λ[x-1])
            self.bear.append(np.degrees(np.arctan2(y, x))%360) #0-360 instead of -180:180
            
       
    def speed(self,**kwargs):
        mps_mph = 2.23694
        mps_kph = 3.6
        if 'mph' in kwargs:
            self.mph = kwargs['mph']
            self.mps = self.mph/mps_mph
            self.kph = self.mps*mps_kph
        if 'kph' in kwargs:
            self.kph = kwargs['kph']
            self.mps = self.kph/mps_kph
            self.mph = self.mps*mps_mph
        if 'mps' in kwargs:
            self.mps = kwargs['mps']
            self.kph = self.mps*mps_kph
            self.mph = self.mps*mps_mph
            
            
    def decimate(self,**kwargs): #todo add variable input parameters such as dist=10km or time = 1hr or numpoints = 20
        if 'Distance' in kwargs:
            distance = kwargs['Distance']
            distanceBetweenPoints = self.distance[1]-self.distance[0] #assumed constant should average or something
            numPoints = self.length*(distanceBetweenPoints/distance)
            numPoints = np.floor(numPoints).astype(int)
        if 'Points' in kwargs:
            numPoints = kwargs['Points']
        if 'Time' in kwargs:
            if hasattr(self, 'mps'):
                print('TODO')
            else:
                raise Exception('no speed defined use x.Speed(mps =y | kph =z | mph =w)')
            numPoints = 10
        else:
            numPoints = 10
       
        print('Decimating to {0} Points'.format(numPoints))   
        ind = np.linspace(0, (self.length-1),numPoints, endpoint=True, retstep=False, dtype=None)
        ind = np.floor(ind)
        ind = ind.astype(int)
        self.len = numPoints
        self.lat = np.array(self.latitude)
        self.lon = np.array(self.longitude)
        self.dist = np.array(self.distance)
        self.lat = self.lat[np.ix_(ind)]
        self.lon = self.lon[np.ix_(ind)]
        self.dist = self.dist[np.ix_(ind)]
        self.__bearingdec__()
        
        
    def getWeatherData(self,apikey='none',units='si',writeF=False):
        self.weatherData = list()
        if hasattr(self, 'len'):
            print('Gathering weather data...')
        else:
            raise Exception('Data not decimated not making API call')

        
        for x in range(0,self.len):
            url='https://api.darksky.net/forecast/{0}/{1},{2}?units={3}'.format(apikey,self.lat[x],self.lon[x],units)
            data = (requests.get(url).content)
            if writeF:
                if not os.path.exists('weatherData'):
                    os.makedirs('weatherData')
                file = open('weatherData/weatherdataDemoTCX{0}.json'.format(x),'wb')     
                file.write(data)
                file.close
            weatherData.append(json.load(data))
        print('Gathered weather data')
    
    
    def loadExistingData(self,location):
        if hasattr(self,'weatherData'):
             raise Exception('Data already exists')
        self.weatherData = list()
                   
        for x in range(0,self.len):
            filename = '{0}{1}.json'.format(location,x)
            #print('Loading' filename)
            with open(filename) as data_file:  
                self.weatherData.append(json.load(data_file))

            

In [ ]:
steve = tcxWeather()
steve.decimate(Points=10)
steve.loadExistingData('weatherData/weatherdataDemoTCX')
#steve.getWeatherData(apikey='',units='si',writeF=True)


In [ ]:
#steve=tcxWeather()
#steve.decimate(Points=10)
pprint(steve.weatherData[0]["hourly"]["data"][0])

In [ ]:
pprint(steve.weatherData[0]["hourly"]["data"][0]["precipIntensity"])
pprint(steve.weatherData[9]["hourly"]["data"][3]["precipIntensity"])